In [1]:
import os
import numpy as np
import librosa as lr
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model("./models/final.model")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
genres = {0: "Hip-Pop", 
          1: "Pop", 
          2: "Folk", 
          3: "Experimental", 
          4: "Rock", 
          5: "International", 
          6: "Electronic", 
          7: "Instrumental"}

In [4]:
input_data = []

for track in os.listdir("./data/input/"):
    if track[-4:] == ".mp3":
        input_track = [track[:-4]]
        
        try:
            y, sr = lr.load(f"./data/input/{track}", sr=None)
            
            features = []
            num_seg = len(y) // (5 * sr)
            
            for i in range(num_seg):
                y_idx = i * (5 * sr)
                y_seg = y[y_idx:y_idx + (5 * sr)]
                
                hl = int((sr / 44100) * 2048)
                
                feature = lr.feature.melspectrogram(y_seg, sr, n_mels=128, hop_length=hl)
                feature = np.array(feature).reshape(-1, 128, 108, 1)
                
                features.append(feature)

            input_track.append(features)
        except Exception as e:
            pass
        
        input_data.append(input_track)

In [5]:
for track in input_data:
    print(f"Track Title: {track[0]:<31} Total Segments: {len(track[1])}")

Track Title: Bon Jovi - Livin' On A Prayer   Total Segments: 49
Track Title: Caravan Palace - Lone Digger    Total Segments: 34
Track Title: Gillicuddy - Springish          Total Segments: 28
Track Title: NF - When I Grow Up             Total Segments: 48
Track Title: Sasha Sloan - Dancing With Your Ghost Total Segments: 39


In [6]:
for track in input_data:
    predicted_genres = {v: 0 for k, v in genres.items()}
    for feature in track[1]:
        prediction = model.predict(feature)
        predicted_genre = genres[np.argmax(prediction)]
        
        predicted_genres[predicted_genre] = predicted_genres[predicted_genre] + 1

    top_genre = [genres[0], 0]
    for k, v in predicted_genres.items():
        if v > top_genre[1] and k != "Experimental":
            top_genre = [k, v]
        
    track.append(top_genre[0])
    track.append(predicted_genres)

In [7]:
for track in input_data:
    print(f"Track Title: {track[0]:<31} Top Genre: {track[2]}")

Track Title: Bon Jovi - Livin' On A Prayer   Top Genre: Rock
Track Title: Caravan Palace - Lone Digger    Top Genre: Electronic
Track Title: Gillicuddy - Springish          Top Genre: Folk
Track Title: NF - When I Grow Up             Top Genre: Hip-Pop
Track Title: Sasha Sloan - Dancing With Your Ghost Top Genre: Pop


In [8]:
for track in input_data:
    print(f"\nTrack Title: {track[0]:<31}")
    
    for k, v in track[3].items():
        print(f" - {k:<13}: {v:}")


Track Title: Bon Jovi - Livin' On A Prayer  
 - Hip-Pop      : 0
 - Pop          : 6
 - Folk         : 0
 - Experimental : 11
 - Rock         : 32
 - International: 0
 - Electronic   : 0
 - Instrumental : 0

Track Title: Caravan Palace - Lone Digger   
 - Hip-Pop      : 3
 - Pop          : 5
 - Folk         : 0
 - Experimental : 6
 - Rock         : 0
 - International: 0
 - Electronic   : 20
 - Instrumental : 0

Track Title: Gillicuddy - Springish         
 - Hip-Pop      : 0
 - Pop          : 6
 - Folk         : 12
 - Experimental : 9
 - Rock         : 0
 - International: 0
 - Electronic   : 0
 - Instrumental : 1

Track Title: NF - When I Grow Up            
 - Hip-Pop      : 19
 - Pop          : 2
 - Folk         : 0
 - Experimental : 17
 - Rock         : 1
 - International: 0
 - Electronic   : 9
 - Instrumental : 0

Track Title: Sasha Sloan - Dancing With Your Ghost
 - Hip-Pop      : 4
 - Pop          : 16
 - Folk         : 8
 - Experimental : 4
 - Rock         : 2
 - International: